# Geocoding Toa Payoh's 2022 Resale Flat Data

### Author: Kok Jim Meng

The output of this data processing is for geospatial feature engineering using ArcPy on the sum weighted distance of the block to the NEA's market and food centres. <br>
Source: <br>
* NEA Market and Food Centre: https://beta.data.gov.sg/datasets/1389/view
* HDB Resale Data: https://beta.data.gov.sg/datasets/189/view
* Convert KML data to SHP: https://mygeodata.cloud/converter/kml-to-shp

For example, in the buffer or catchment of 400m of each block, how many market and food centres are in the catchment and what's the distance between the block and each of the market/food centre?

Eg. Blk 126 has a market within 50m and a food centre within 120m. The sum weighted value for Blk 126 will be:

<center>$1/50 + 1/120 = 0.028$</center>

### Note:
* Data as of 16 Sept 2023 
* Due to high intensification of data, I only select 2022's resale flat data and target at Toa Payoh town
* Not all blocks' addresses are able to geocode using Nominatim and geopy due to it's open-sourced
* The data is just a sample data for showing the example of geospatial feature engineering using ArcPy in another notebook

In [1]:
# Import all the libraries
import glob
import pandas as pd
import numpy as np
import json
import requests
import datetime
from geopy.geocoders import Nominatim

In [2]:
# Read the data
df = pd.concat([pd.read_csv(f, parse_dates=['month']) for f in glob.glob("../Data/raw/*.csv")],
               ignore_index=True)
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2015-01-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0
1,2015-01-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0
2,2015-01-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0
3,2015-01-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0
4,2015-01-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0


In [3]:
# Get the year
df["year"] = df["month"].dt.year
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,year
0,2015-01-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,2015
1,2015-01-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,2015
2,2015-01-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,2015
3,2015-01-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,2015
4,2015-01-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,2015
...,...,...,...,...,...,...,...,...,...,...,...,...
198986,2023-09-01,YISHUN,5 ROOM,511B,YISHUN ST 51,04 TO 06,113.0,Improved,2017,93 years,620000.0,2023
198987,2023-09-01,YISHUN,5 ROOM,504D,YISHUN ST 51,07 TO 09,112.0,Improved,2016,91 years 07 months,685000.0,2023
198988,2023-09-01,YISHUN,EXECUTIVE,392,YISHUN AVE 6,07 TO 09,142.0,Apartment,1988,63 years 09 months,740000.0,2023
198989,2023-09-01,YISHUN,EXECUTIVE,355,YISHUN RING RD,04 TO 06,150.0,Maisonette,1988,64 years 03 months,875000.0,2023


In [4]:
# Filter to extract 2022's resale HDB blocks
resale_df_2022 = df[df["year"]==2022]
resale_df_2022

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,year
153829,2022-01-01,ANG MO KIO,2 ROOM,323,ANG MO KIO AVE 3,07 TO 09,44.0,Improved,1977,54 years 05 months,245000.0,2022
153830,2022-01-01,ANG MO KIO,3 ROOM,320,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1977,54 years 05 months,358000.0,2022
153831,2022-01-01,ANG MO KIO,3 ROOM,225,ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,1978,55 years 01 month,355000.0,2022
153832,2022-01-01,ANG MO KIO,3 ROOM,331,ANG MO KIO AVE 1,07 TO 09,68.0,New Generation,1981,58 years,338000.0,2022
153833,2022-01-01,ANG MO KIO,3 ROOM,534,ANG MO KIO AVE 10,07 TO 09,82.0,New Generation,1980,57 years 02 months,420000.0,2022
...,...,...,...,...,...,...,...,...,...,...,...,...
180544,2022-12-01,YISHUN,EXECUTIVE,355,YISHUN RING RD,07 TO 09,146.0,Maisonette,1988,65 years,845000.0,2022
180545,2022-12-01,YISHUN,EXECUTIVE,643,YISHUN ST 61,07 TO 09,142.0,Apartment,1987,63 years 10 months,782000.0,2022
180546,2022-12-01,YISHUN,EXECUTIVE,643,YISHUN ST 61,07 TO 09,142.0,Apartment,1987,63 years 10 months,770888.0,2022
180547,2022-12-01,YISHUN,EXECUTIVE,606,YISHUN ST 61,10 TO 12,146.0,Maisonette,1987,64 years,788000.0,2022


In [5]:
# Filter to get Toa Payoh's
tpy_resale_df_2022 = resale_df_2022[resale_df_2022["town"]=="TOA PAYOH"]
tpy_resale_df_2022

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,year
155901,2022-01-01,TOA PAYOH,3 ROOM,126,LOR 1 TOA PAYOH,16 TO 18,67.0,Improved,1970,47 years 01 month,338000.0,2022
155902,2022-01-01,TOA PAYOH,3 ROOM,157,LOR 1 TOA PAYOH,04 TO 06,67.0,Improved,1971,48 years 07 months,345000.0,2022
155903,2022-01-01,TOA PAYOH,3 ROOM,173,LOR 1 TOA PAYOH,10 TO 12,66.0,Improved,1972,49 years 07 months,308000.0,2022
155904,2022-01-01,TOA PAYOH,3 ROOM,114,LOR 1 TOA PAYOH,01 TO 03,65.0,Improved,1967,44 years 06 months,300000.0,2022
155905,2022-01-01,TOA PAYOH,3 ROOM,158,LOR 1 TOA PAYOH,04 TO 06,65.0,Improved,1972,49 years 01 month,310000.0,2022
...,...,...,...,...,...,...,...,...,...,...,...,...
180183,2022-12-01,TOA PAYOH,5 ROOM,250,KIM KEAT LINK,07 TO 09,128.0,Improved,1989,65 years 02 months,738000.0,2022
180184,2022-12-01,TOA PAYOH,5 ROOM,139B,LOR 1A TOA PAYOH,34 TO 36,114.0,DBSS,2012,88 years 06 months,1160000.0,2022
180185,2022-12-01,TOA PAYOH,5 ROOM,82,LOR 4 TOA PAYOH,01 TO 03,122.0,Improved,1996,72 years 11 months,830000.0,2022
180186,2022-12-01,TOA PAYOH,5 ROOM,229,LOR 8 TOA PAYOH,04 TO 06,117.0,Standard,1976,52 years 11 months,582888.0,2022


In [6]:
# Get the relevant cols
unique_tpy_resale_buildings_2022_df = tpy_resale_df_2022[["block", "street_name", "lease_commence_date"]]
unique_tpy_resale_buildings_2022_df

,block,street_name,lease_commence_date
155901,126,LOR 1 TOA PAYOH,1970
155902,157,LOR 1 TOA PAYOH,1971
155903,173,LOR 1 TOA PAYOH,1972
155904,114,LOR 1 TOA PAYOH,1967
155905,158,LOR 1 TOA PAYOH,1972
...,...,...,...
180183,250,KIM KEAT LINK,1989
180184,139B,LOR 1A TOA PAYOH,2012
180185,82,LOR 4 TOA PAYOH,1996
180186,229,LOR 8 TOA PAYOH,1976


In [7]:
# Get the remaining lease of years since 2023
unique_tpy_resale_buildings_2022_df["remaining_lease_since_2023"] = datetime.datetime.now().year - unique_tpy_resale_buildings_2022_df["lease_commence_date"]
unique_tpy_resale_buildings_2022_df

C:\Users\jimme\AppData\Local\Temp\ipykernel_16040\751372457.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_tpy_resale_buildings_2022_df["remaining_lease_since_2023"] = datetime.datetime.now().year - unique_tpy_resale_buildings_2022_df["lease_commence_date"]


,block,street_name,lease_commence_date,remaining_lease_since_2023
155901,126,LOR 1 TOA PAYOH,1970,53
155902,157,LOR 1 TOA PAYOH,1971,52
155903,173,LOR 1 TOA PAYOH,1972,51
155904,114,LOR 1 TOA PAYOH,1967,56
155905,158,LOR 1 TOA PAYOH,1972,51
...,...,...,...,...
180183,250,KIM KEAT LINK,1989,34
180184,139B,LOR 1A TOA PAYOH,2012,11
180185,82,LOR 4 TOA PAYOH,1996,27
180186,229,LOR 8 TOA PAYOH,1976,47


In [8]:
# Remove the duplicate of blocks
unique_tpy_resale_buildings_2022_df = unique_tpy_resale_buildings_2022_df.drop_duplicates()
unique_tpy_resale_buildings_2022_df

,block,street_name,lease_commence_date,remaining_lease_since_2023
155901,126,LOR 1 TOA PAYOH,1970,53
155902,157,LOR 1 TOA PAYOH,1971,52
155903,173,LOR 1 TOA PAYOH,1972,51
155904,114,LOR 1 TOA PAYOH,1967,56
155905,158,LOR 1 TOA PAYOH,1972,51
...,...,...,...,...
175894,131,POTONG PASIR AVE 1,1984,39
177896,159,LOR 1 TOA PAYOH,1972,51
177907,94,LOR 4 TOA PAYOH,1967,56
180154,27,TOA PAYOH EAST,1971,52


In [9]:
# Get the address
unique_tpy_resale_buildings_2022_df["address"] = unique_tpy_resale_buildings_2022_df["block"]
                                                + " "
                                                + unique_tpy_resale_buildings_2022_df["street_name"]
unique_tpy_resale_buildings_2022_df = unique_tpy_resale_buildings_2022_df.reset_index()
unique_tpy_resale_buildings_2022_df

C:\Users\jimme\AppData\Local\Temp\ipykernel_16040\861515259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_tpy_resale_buildings_2022_df["address"] = unique_tpy_resale_buildings_2022_df["block"] + " " + unique_tpy_resale_buildings_2022_df["street_name"]


,index,block,street_name,lease_commence_date,remaining_lease_since_2023,address
0,155901,126,LOR 1 TOA PAYOH,1970,53,126 LOR 1 TOA PAYOH
1,155902,157,LOR 1 TOA PAYOH,1971,52,157 LOR 1 TOA PAYOH
2,155903,173,LOR 1 TOA PAYOH,1972,51,173 LOR 1 TOA PAYOH
3,155904,114,LOR 1 TOA PAYOH,1967,56,114 LOR 1 TOA PAYOH
4,155905,158,LOR 1 TOA PAYOH,1972,51,158 LOR 1 TOA PAYOH
...,...,...,...,...,...,...
230,175894,131,POTONG PASIR AVE 1,1984,39,131 POTONG PASIR AVE 1
231,177896,159,LOR 1 TOA PAYOH,1972,51,159 LOR 1 TOA PAYOH
232,177907,94,LOR 4 TOA PAYOH,1967,56,94 LOR 4 TOA PAYOH
233,180154,27,TOA PAYOH EAST,1971,52,27 TOA PAYOH EAST


In [11]:
# Remove the extra cols
unique_tpy_resale_buildings_2022_df = unique_tpy_resale_buildings_2022_df.drop(columns=['index',
                                                                                        'block',
                                                                                        'street_name',
                                                                                        'lease_commence_date'])
unique_tpy_resale_buildings_2022_df

,remaining_lease_since_2023,address
0,53,126 LOR 1 TOA PAYOH
1,52,157 LOR 1 TOA PAYOH
2,51,173 LOR 1 TOA PAYOH
3,56,114 LOR 1 TOA PAYOH
4,51,158 LOR 1 TOA PAYOH
...,...,...
230,39,131 POTONG PASIR AVE 1
231,51,159 LOR 1 TOA PAYOH
232,56,94 LOR 4 TOA PAYOH
233,52,27 TOA PAYOH EAST


In [13]:
# Geocoding
geolocator = Nominatim(user_agent="my_app")  # using open street map API
unique_tpy_resale_buildings_2022_df['geocode'] = unique_tpy_resale_buildings_2022_df['address'].apply(geolocator.geocode)

In [15]:
# Check how's the geocoding has processed
unique_tpy_resale_buildings_2022_df

,remaining_lease_since_2023,address,geocode
0,53,126 LOR 1 TOA PAYOH,None
1,52,157 LOR 1 TOA PAYOH,None
2,51,173 LOR 1 TOA PAYOH,None
3,56,114 LOR 1 TOA PAYOH,None
4,51,158 LOR 1 TOA PAYOH,None
...,...,...,...
230,39,131 POTONG PASIR AVE 1,"(131, Potong Pasir Avenue 1, Toa Payoh, Southe..."
231,51,159 LOR 1 TOA PAYOH,None
232,56,94 LOR 4 TOA PAYOH,None
233,52,27 TOA PAYOH EAST,"(27, Toa Payoh East, Toa Payoh, Singapore, Cen..."


In [16]:
# Remove the empty geocoded blocks 😣
unique_tpy_resale_buildings_2022_df = unique_tpy_resale_buildings_2022_df.dropna()
unique_tpy_resale_buildings_2022_df

,remaining_lease_since_2023,address,geocode
9,53,116 LOR 2 TOA PAYOH,"(Toa Payoh, Singapore, Central, Singapore, (1...."
18,39,101 POTONG PASIR AVE 1,"(101, Potong Pasir Avenue 1, Market Square, To..."
19,39,104 POTONG PASIR AVE 1,"(104, Potong Pasir Avenue 1, Market Square, To..."
20,39,115 POTONG PASIR AVE 1,"(115, Potong Pasir Avenue 1, Toa Payoh, Southe..."
21,39,140 POTONG PASIR AVE 3,"(140, Potong Pasir Avenue 3, Toa Payoh, Singap..."
...,...,...,...
226,35,20 JOO SENG RD,"(Joo Seng Road, Tai Seng, Joo Seng, Southeast,..."
227,39,124 POTONG PASIR AVE 1,"(124, Potong Pasir Avenue 1, Market Square, To..."
229,39,135 POTONG PASIR AVE 3,"(135, Potong Pasir Avenue 3, Toa Payoh, Singap..."
230,39,131 POTONG PASIR AVE 1,"(131, Potong Pasir Avenue 1, Toa Payoh, Southe..."


In [17]:
# Get the latitude and longitude
unique_tpy_resale_buildings_2022_df['latitude'] = [g.latitude for g in unique_tpy_resale_buildings_2022_df.geocode]
unique_tpy_resale_buildings_2022_df['longitude'] = [g.longitude for g in unique_tpy_resale_buildings_2022_df.geocode]

C:\Users\jimme\AppData\Local\Temp\ipykernel_16040\3925462539.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_tpy_resale_buildings_2022_df['latitude'] = [g.latitude for g in unique_tpy_resale_buildings_2022_df.geocode]
C:\Users\jimme\AppData\Local\Temp\ipykernel_16040\3925462539.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_tpy_resale_buildings_2022_df['longitude'] = [g.longitude for g in unique_tpy_resale_buildings_2022_df.geocode]


In [18]:
# Output
unique_tpy_resale_buildings_2022_df

,remaining_lease_since_2023,address,geocode,latitude,longitude
9,53,116 LOR 2 TOA PAYOH,"(Toa Payoh, Singapore, Central, Singapore, (1....",1.335391,103.849741
18,39,101 POTONG PASIR AVE 1,"(101, Potong Pasir Avenue 1, Market Square, To...",1.333029,103.868518
19,39,104 POTONG PASIR AVE 1,"(104, Potong Pasir Avenue 1, Market Square, To...",1.334356,103.869273
20,39,115 POTONG PASIR AVE 1,"(115, Potong Pasir Avenue 1, Toa Payoh, Southe...",1.337438,103.862638
21,39,140 POTONG PASIR AVE 3,"(140, Potong Pasir Avenue 3, Toa Payoh, Singap...",1.333839,103.866940
...,...,...,...,...,...
226,35,20 JOO SENG RD,"(Joo Seng Road, Tai Seng, Joo Seng, Southeast,...",1.337641,103.882505
227,39,124 POTONG PASIR AVE 1,"(124, Potong Pasir Avenue 1, Market Square, To...",1.335305,103.865496
229,39,135 POTONG PASIR AVE 3,"(135, Potong Pasir Avenue 3, Toa Payoh, Singap...",1.334683,103.866851
230,39,131 POTONG PASIR AVE 1,"(131, Potong Pasir Avenue 1, Toa Payoh, Southe...",1.334365,103.865327


In [19]:
# Save the output into a file
unique_tpy_resale_buildings_2022_df.to_csv("../Data/processed/unique_tpy_resale_buildings_2022_df.csv",
                                           index=False)